### Data processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [2]:
data = pd.read_json('train.json')

In [3]:
data['band_1'] = data['band_1'].apply(lambda x: np.array(x).reshape(75,75))
data['band_2'] = data['band_2'].apply(lambda x: np.array(x).reshape(75,75))

In [4]:
train = data.sample(frac=0.8)
val = data[~data.isin(train)].dropna()

In [5]:
band_1_tr = np.concatenate([im for im in train['band_1']]).reshape(-1, 75, 75)
band_2_tr = np.concatenate([im for im in train['band_2']]).reshape(-1, 75, 75)
full_img_tr = np.stack([band_1_tr, band_2_tr], axis=1)

In [25]:
full_img_tr.shape

(1283, 2, 75, 75)

In [6]:
band_1_val = np.concatenate([im for im in val['band_1']]).reshape(-1, 75, 75)
band_2_val = np.concatenate([im for im in val['band_2']]).reshape(-1, 75, 75)
full_img_val = np.stack([band_1_val, band_2_val], axis=1)

### Torch libraries

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam, Adadelta
import torch.nn.init as init
from torch.utils.data import TensorDataset, DataLoader

### Data loading

In [27]:
y_train = train['is_iceberg'].values
y_train = np.append(y_train.reshape(-1,1), np.zeros([len(y_train),1]),1)
for i in range(len(y_train)):
    if y_train[i,0] == 0:
        y_train[i,1] = 1
#y_train

In [28]:
y_val = val['is_iceberg'].values
y_val = np.append(y_val.reshape(-1,1), np.zeros([len(y_val),1]),1)
for i in range(len(y_val)):
    if y_val[i,0] == 0:
        y_val[i,1] = 1
y_val

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,

In [29]:
train_imgs = torch.from_numpy(full_img_tr).float()
train_targets = torch.from_numpy(y_train).long()
train_dataset = TensorDataset(train_imgs, train_targets)

In [30]:
train_dataset[0][0].size()

torch.Size([2, 75, 75])

In [31]:
val_imgs = torch.from_numpy(full_img_val).float()
val_targets = torch.from_numpy(y_val).long()
val_dataset = TensorDataset(val_imgs, val_targets)

### Network

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batch = nn.BatchNorm2d(2)
        self.conv1 = nn.Conv2d(2, 12, kernel_size=5, padding=1)
        init.xavier_normal(self.conv1.weight, gain = np.sqrt(2.0))
        init.constant(self.conv1.bias, 0.1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 20, kernel_size=5, padding=1)
        init.xavier_normal(self.conv2.weight, gain = np.sqrt(2.0))
        init.constant(self.conv2.bias, 0.1)
        self.fc1 = nn.Linear(20 * 17 * 17, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        x = self.batch(x)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.Softmax(self.fc3(x))
        return x

In [34]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 6, kernel_size=5, padding=0)
        init.xavier_normal(self.conv1.weight, gain = np.sqrt(2.0))
        init.constant(self.conv1.bias, 0.1)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5, padding=0, stride = 2)
        init.xavier_normal(self.conv2.weight, gain = np.sqrt(2.0))
        init.constant(self.conv2.bias, 0.1)
        self.conv3 = nn.Conv2d(12, 24, kernel_size=5, padding=0, stride = 2)
        init.xavier_normal(self.conv3.weight, gain = np.sqrt(2.0))
        init.constant(self.conv3.bias, 0.1)
        self.fc1 = nn.Linear(24 * 15 * 15, 120)
        self.fc2 = nn.Linear(120, 2)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [62]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(2, 6, kernel_size=5, padding=0),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 12, kernel_size=5, padding=0, stride = 2),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Conv2d(12, 24, kernel_size=5, padding=0, stride = 2),
            nn.ReLU())
        self.fc1 = nn.Sequential(
                nn.Linear(15*15*24, 200),
                nn.ReLU())
        self.fc2 = nn.Linear(200,2)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [79]:
np.sqrt(95256/24)

63.0

In [35]:
model = Net()

In [36]:
model

Net (
  (conv1): Conv2d(2, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(2, 2))
  (conv3): Conv2d(12, 24, kernel_size=(5, 5), stride=(2, 2))
  (fc1): Linear (5400 -> 120)
  (fc2): Linear (120 -> 2)
)

### Training

In [14]:
epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = Adadelta(model.parameters(), lr = 0.01)

In [15]:
class AverageMeter(object):
######     Computes and stores the average and current value
    def __init__(self, window_size=None):
        self.length = 0
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.window_size = window_size
    def reset(self):
        self.length = 0
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        if self.window_size and (self.count >= self.window_size):
            self.reset()
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count

In [16]:
def accuracy(y_true, y_pred):
    y_true = y_true.float()
    _, y_pred = torch.max(y_pred, dim=-1)
    return (y_pred.float() == y_true).float().mean()

In [17]:
from tqdm import tqdm_notebook
def fit(train, val, epochs, batch_size):
    print('train on {} images validate on {} images'.format(len(train), len(val)))
    model.train()
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False)
    for epoch in tqdm_notebook(range(epochs), total=epochs):
        running_loss = AverageMeter()
        running_accuracy = AverageMeter()
        val_loss_meter = AverageMeter()
        val_acc_meter = AverageMeter()
        pbar = tqdm_notebook(train_loader, total=len(train_loader))
        for data, target in pbar:
            data, target = Variable(data), Variable(target)
            output = model(data)
            loss = criterion(output, target)
            acc = accuracy(target.data, output.data)
            running_loss.update(loss.data[0])
            running_accuracy.update(acc)
            pbar.set_description("[ loss: {:.4f} | acc: {:.4f} ] ".format(running_loss.avg, running_accuracy.avg))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print("[ loss: {:.4f} | acc: {:.4f} ] ".format(running_loss.avg, running_accuracy.avg))
        for val_data, val_target in val_loader:
            val_data, val_target = Variable(val_data), Variable(val_target)
            output = model(val_data)
            val_loss = criterion(output, val_target)
            val_acc = accuracy(val_target.data, output.data)
            val_loss_meter.update(val_loss.data[0])
            val_acc_meter.update(val_acc)
        pbar.set_description("[ loss: {:.4f} | acc: {:.4f} | vloss: {:.4f} | vacc: {:.4f} ] ".format(running_loss.avg, running_accuracy.avg, val_loss_meter.avg, val_acc_meter.avg))
        print("[ loss: {:.4f} | acc: {:.4f} | vloss: {:.4f} | vacc: {:.4f} ] ".format(running_loss.avg, running_accuracy.avg, val_loss_meter.avg, val_acc_meter.avg))

In [37]:
fit(train_dataset, val_dataset, 3, 32)

train on 1283 images validate on 321 images


RuntimeError: multi-target not supported at /opt/conda/conda-bld/pytorch_1501969512886/work/pytorch-0.1.12/torch/lib/THNN/generic/ClassNLLCriterion.c:20

In [61]:
list(model.conv2.parameters())

[Parameter containing:
 (0 ,0 ,.,.) = 
  -1.4889e-01  4.5710e-02 -5.0143e-02 -2.7000e-03 -6.6145e-04
   7.8826e-02 -1.2682e-01 -3.5409e-02  5.9538e-02 -1.0565e-01
  -1.3036e-01  1.2130e-02 -7.2381e-02 -7.7606e-02  4.9357e-03
   4.4284e-02  3.7420e-02  5.4858e-02 -4.7745e-02  3.3850e-02
  -1.6332e-02 -4.7159e-03 -5.0119e-02 -5.2910e-02  3.1945e-02
 
 (0 ,1 ,.,.) = 
  -9.8006e-02  2.1212e-02 -6.9720e-03  1.0578e-01 -7.4700e-03
   3.5693e-02  1.2701e-01 -4.9049e-02 -1.2123e-02 -9.4965e-02
   5.7547e-02  4.2047e-03 -1.1264e-01  4.8172e-02  3.4292e-02
  -6.5764e-02  2.4009e-02  8.4801e-02  5.0087e-02  1.4401e-02
   5.4766e-02  5.3015e-02 -9.1106e-02  1.1502e-02 -4.2703e-02
 
 (0 ,2 ,.,.) = 
   2.9371e-02 -3.8912e-02 -6.3785e-02 -3.7863e-02 -1.0076e-01
  -5.1366e-02 -7.2622e-04 -5.0236e-02  4.2120e-02  3.8804e-02
  -6.6644e-02 -1.6767e-02 -3.2754e-02  8.4586e-02  4.6213e-02
  -9.8049e-02  8.7449e-04 -8.4819e-02  1.1458e-01 -4.3753e-02
   4.5621e-02  4.2223e-02  1.8311e-01  5.9490e-03 -1.4102

### Test Submission

In [ ]:
test = pd.read_json('test.json')